## Cleaning relevance dataset and query logs

In [221]:
import pandas as pd

relevance_df_1 = pd.read_excel("./data/QL_IT_oct_7_exp_on_results.xlsx")
relevance_df_2 = pd.read_excel("./data/QL_IT_oct_8_exp_on_results.xlsx")
relevance_df_3 = pd.read_excel("./data/QL_IT_oct_10_exp_on_results.xlsx")
rel_df_other = pd.read_excel("./data/QL_IT_exp_on_results.xlsx")
emotional_rel_df = pd.read_excel("./data/QL_IT_oct__exp_on_results.xlsx")
relevance_df = pd.concat([relevance_df_1, relevance_df_2, relevance_df_3, rel_df_other, emotional_rel_df])
# relevance_df = pd.concat([relevance_df_1, relevance_df_2, relevance_df_3])
relevance_df["user_query"] = [query.strip() for query in relevance_df["user_query"]] # removing leading/trailing white spaces in queries
relevance_df = relevance_df[['session_id', 'user_query', 'url', 'source', 'title', 'snippet',
       'position', 'TextSample', 'TextSample-Tokenized',
       'tagged_TextSample-Tokenized']]
relevance_df.drop_duplicates(inplace=True)
relevance_df.reset_index(drop=True, inplace=True)
relevance_df.to_excel("./data/search_results.xlsx", index=False)

clicked_df_1 = pd.read_excel("./data/QL_Oct 7_clicked.xlsx")
clicked_df_2 = pd.read_excel("./data/QL_Oct 8_clicked.xlsx")
clicked_df_3 = pd.read_excel("./data/QL_Oct 10_clicked.xlsx")
clicked_df = pd.concat([clicked_df_1, clicked_df_2, clicked_df_3])
clicked_df["Query"] = [query.strip() for query in clicked_df["Query"]] # removing leading/trailing white spaces in queries
clicked_df.drop_duplicates(inplace=True)
clicked_df.reset_index(drop=True, inplace=True)
clicked_df.to_excel("./data/clicked_results.xlsx", index=False)

In [222]:
print(len(clicked_df_1["Query"].unique()), len(clicked_df_2["Query"].unique()), len(clicked_df_3["Query"].unique())) 
print(len(clicked_df["Query"].unique()))
print(len(relevance_df_1["user_query"].unique()), len(relevance_df_2["user_query"].unique()), len(relevance_df_3["user_query"].unique()), len(rel_df_other["user_query"].unique()), len(emotional_rel_df["user_query"].unique()))
print(len(relevance_df["user_query"].unique()))

40 16 18
70
45 22 20 211 82
282


In [223]:
relevance_df.columns

Index(['session_id', 'user_query', 'url', 'source', 'title', 'snippet',
       'position', 'TextSample', 'TextSample-Tokenized',
       'tagged_TextSample-Tokenized'],
      dtype='object')

In [224]:
relevance_df.head(1)

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,R,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,1,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de..."


In [225]:
clicked_df.head(1)

,Query,url,title,position
0,le conseguenze negative della distruzione dell...,http://www.eniscuola.net/argomento/foresta-tro...,Conseguenze della deforestazione - eniscuola.net,5


In [226]:
len(relevance_df), len(clicked_df)

(3432, 111)

In [227]:
def get_search_session_stats(relevance_df, df_name):
    session_stats = []

    for session_id in relevance_df["session_id"].unique():
        search_session = relevance_df.loc[relevance_df["session_id"]==session_id]
        for query in search_session["user_query"].unique():
            search_results = search_session.loc[search_session["user_query"]==query]
            session_stats.append([session_id, query, len(search_results), len(search_results["url"].unique())])

    df = pd.DataFrame(session_stats, columns=["session_id", "user_query", "num_results", "num_unique_urls"])
    df.to_excel("./data/search_session_stats_"+df_name+".xlsx", index=False)
    df

In [228]:
get_search_session_stats(relevance_df, df_name="relevance_df")

In [229]:
print(len(relevance_df["url"].unique()), len(relevance_df["TextSample"].unique()), len(relevance_df["title"].unique()), len(relevance_df["snippet"].unique()))

1556 1995 1612 1965


In [230]:
rows_to_remove = []
full_session_removed = []

for session_id in relevance_df["session_id"].unique():
    search_session = relevance_df.loc[relevance_df["session_id"]==session_id] # search results for a session id
    print("Session ID: ", session_id)
    duplicate_counter = 0
    for query in search_session["user_query"].unique():
        search_results = relevance_df.loc[(relevance_df["session_id"]==session_id) & (relevance_df["user_query"]==query)] # search results for a query within a session
        if len(search_results) != len(search_results["url"].unique()): # check if the same webpage appeared more than once in the SERP for query during a given session
            print("\tQuery with duplicate URLs in SERP: ", query)
            for url in search_results["url"].unique():
                if len(relevance_df.loc[(relevance_df["session_id"]==session_id) & (relevance_df["user_query"]==query) & (relevance_df["url"]==url)]) > 1:
                    rows_to_remove += list(relevance_df.loc[(relevance_df["session_id"]==session_id) & (relevance_df["user_query"]==query) & (relevance_df["url"]==url)].index)
                    print("\t\tDuplicate URL: ", url, list(relevance_df.loc[(relevance_df["session_id"]==session_id) & (relevance_df["user_query"]==query) & (relevance_df["url"]==url)].index))
                    duplicate_counter += len(list(relevance_df.loc[(relevance_df["session_id"]==session_id) & (relevance_df["user_query"]==query) & (relevance_df["url"]==url)].index))
    print(duplicate_counter," results to be removed from session: ", session_id)
    if duplicate_counter==len(search_session):
        print("*******************************************")
        print("WARNING!! Full session ", session_id, " has to be removed")
        full_session_removed.append(session_id)
        print("*******************************************")
    print("Updated list of rows to remove: ", rows_to_remove)
    print()

print("Number of sessions to be removed completely: ", len(full_session_removed))
    
relevance_df_cleaned = relevance_df.drop(relevance_df.index[rows_to_remove])
relevance_df_cleaned.reset_index(inplace=True, drop=True)
relevance_df_cleaned.to_excel("./data/search_results_cleaned.xlsx", index=False)
relevance_df_cleaned.head(2)

Session ID:  393
0  results to be removed from session:  393
Updated list of rows to remove:  []

Session ID:  395
	Query with duplicate URLs in SERP:  Dove si trova l isola di plastica
		Duplicate URL:  https://www.repubblica.it/ambiente/2018/03/22/news/l_isola_di_plastica_del_pacifico_e_sempre_piu_enorme-191962940/ [11, 20]
		Duplicate URL:  http://www.saperescienza.it/rubriche/geologia/l-isola-di-plastica-18-07-2014/200-l-isola-di-plastica-18-07-2014 [16, 21]
	Query with duplicate URLs in SERP:  Il ghiacciaio piÃ…Â« in via di estinzione
		Duplicate URL:  https://it.m.wikipedia.org/wiki/Ghiacciaio_del_Calderone [33, 34]
	Query with duplicate URLs in SERP:  Le regole per tenere le spiagge pulite
		Duplicate URL:  https://www.105.net/gallery/tutto-news/249890/10-consigli-per-mantenere-le-spiagge-pulite.html [43, 52]
		Duplicate URL:  https://www.ecoo.it/articolo/spiagge-italiane-ecco-le-regole-per-trattarle-bene/21287/ [44, 45]
		Duplicate URL:  https://www.laleggepertutti.it/25788_spi

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,R,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,1,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de..."
1,393,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,NR,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,2,La deforestazione in Amazzonia - Foresta Amazz...,"['La', 'deforestazione', 'in', 'Amazzonia', '-...","[('La', 'NNP'), ('deforestazione', 'NN'), ('in..."


In [231]:
get_search_session_stats(relevance_df=relevance_df_cleaned, df_name="relevance_df_cleaned")

In [232]:
print(len(rows_to_remove), len(relevance_df)-len(relevance_df_cleaned))

314 314


In [233]:
print(len(relevance_df_cleaned["session_id"].unique()), len(relevance_df["session_id"].unique()))

94 94


## Combining query log and relevance judgments into one dataset

In [234]:
import pandas as pd

relevance_df = pd.read_excel("./data/search_results_cleaned.xlsx")
relevance_df.head()

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,R,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,1,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de..."
1,393,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,NR,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,2,La deforestazione in Amazzonia - Foresta Amazz...,"['La', 'deforestazione', 'in', 'Amazzonia', '-...","[('La', 'NNP'), ('deforestazione', 'NN'), ('in..."
2,393,le conseguenze negative della distruzione dell...,https://www.elledecor.com/it/lifestyle/a280666...,R,Le conseguenze degli incendi della foresta ama...,Il Brasile sta vivendo la piÃ¹ grande ondata d...,3,Le conseguenze degli incendi della foresta ama...,"['Le', 'conseguenze', 'degli', 'incendi', 'del...","[('Le', 'NNP'), ('conseguenze', 'NN'), ('degli..."
3,393,le conseguenze negative della distruzione dell...,https://www.informazioneambiente.it/deforestaz...,NR,Cause e conseguenze della deforestazione ambie...,Le conseguenze della deforestazione. Ma a cosa...,4,Cause e conseguenze della deforestazione ambie...,"['Cause', 'e', 'conseguenze', 'della', 'defore...","[('Cause', 'NNP'), ('e', 'NN'), ('conseguenze'..."
4,393,le conseguenze negative della distruzione dell...,http://www.eniscuola.net/argomento/foresta-tro...,R,Conseguenze della deforestazione - eniscuola.net,Conseguenze della deforestazione. Le foreste h...,5,Conseguenze della deforestazione - eniscuola.n...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de..."


In [235]:
weird_ones = 0
weird_queries_count = 0
weird_queries = []
weird_titles = []

for query in clicked_df["Query"].unique():
    weird_query_flag = 0
    temp = clicked_df.loc[clicked_df["Query"]==query]
    for _, row in temp.iterrows():
        user_query = row["Query"]
        assert user_query==query
        clicked_url = row["url"]
        clicked_title = row["title"]
        clicked_position = row["position"]
        clicked_results = clicked_df.loc[(clicked_df["Query"]==user_query)&(clicked_df["url"]==clicked_url)&(clicked_df["title"]==clicked_title)&(clicked_df["position"]==clicked_position)]
        if len(clicked_results)>1:
            print(clicked_results)
            break
        found_result = relevance_df.loc[(relevance_df["user_query"]==user_query) & (relevance_df["title"] ==clicked_title) & (relevance_df["url"]==clicked_url) & (relevance_df["position"]==clicked_position)]
        if len(found_result) > 1:
            if len(found_result)!=len(found_result["session_id"].unique()):
                weird_query_flag = 1
            # print(found_result["session_id"].unique())
            # print(user_query)
            # print(clicked_title)
            # print(clicked_url)
            # print(clicked_position)
            # print(found_result["snippet"].unique())
            # print(found_result["TextSample"].unique())
            weird_ones += 1
            weird_titles.append(clicked_title)
    if weird_query_flag == 1:
        weird_queries_count += 1
        weird_queries.append(query)

print(weird_queries_count)
for query in weird_queries:
    print(query)

0


In [236]:
clicked_df.head()

,Query,url,title,position
0,le conseguenze negative della distruzione dell...,http://www.eniscuola.net/argomento/foresta-tro...,Conseguenze della deforestazione - eniscuola.net,5
1,BiodiversitÃƒÂ¡,http://www.globalissues.org/article/170/why-is...,Why Is Biodiversity Important? Who Cares? â€” ...,9
2,perchÃƒÂ© gli orsi polari si stanno estinguend...,http://www.ilcambiamento.it/articoli/orsi_pola...,"Orsi polari, Ã¨ allarme estinzione. Lo conferm...",5
3,Lisola di plastica cosa ÃƒÂ¨,http://www.lascimmiaviaggiatrice.it/isola-di-p...,isola di plastica che cos'Ã¨ e perchÃ¨ non puÃ...,3
4,Quanto psa lisola di plastica,http://www.meteoweb.eu/2018/03/inquinamento-oc...,Inquinamento: la piÃ¹ grande isola ... - meteo...,6


In [237]:
relevance_df.head(1)

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,R,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,1,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de..."


In [238]:
print(len(relevance_df), len(clicked_df), len(relevance_df.loc[relevance_df["user_query"].isin(clicked_df["Query"].unique())]))
print(len(relevance_df["user_query"]), len(clicked_df["Query"]))
print(len(relevance_df["user_query"].unique()), len(clicked_df["Query"].unique()))
print(clicked_df["Query"].value_counts())

3118 111 670
3118 111
282 70
Query
Di cosa si occupa il WWF ?                                         4
Isola di plastica                                                  4
Qual ÃƒÂ© la causa della riduzione del lago Aral                   3
Conseguenze negative della distruzione della foresta amazzonica    3
perchÃƒÂ© gli orsi polari si stanno estinguendo sito babbanas      3
                                                                  ..
Peso bottiglia di plastica                                         1
spiagge bandiera blu 2019                                          1
Spiagge con la bandiera della pulizia                              1
Spiagge con bandiera blu                                           1
CosÃƒÂ¨ Ã„Â¾isola di plastica                                      1
Name: count, Length: 70, dtype: int64


In [239]:
rel_df_queries = list(relevance_df["user_query"].unique())
click_df_queries = list(clicked_df["Query"].unique())

count = 0
for query in click_df_queries:
    if query not in rel_df_queries:
        print(query)
        count+=1

print("Number of queries with no relevance judgments: ", count)

Number of queries with no relevance judgments:  0


In [240]:
missing_clicked_result = 0

for _, row in clicked_df.iterrows():
    query = row["Query"]
    title = row["title"]
    url = row["url"]
    position = row["position"]

    search_result = relevance_df.loc[(relevance_df["user_query"]==query) & (relevance_df["title"]==title) & (relevance_df["url"]==url) & (relevance_df["position"]==position)]
    if len(search_result) == 0:
        missing_clicked_result +=1 
        print(row, end="\n\n")

print("Number of clicked results missing from relevance dataset: ", missing_clicked_result)

Query                                         BiodiversitÃƒÂ¡
url         http://www.globalissues.org/article/170/why-is...
title       Why Is Biodiversity Important? Who Cares? â€” ...
position                                                    9
Name: 1, dtype: object

Query                            Lisola di plastica cosa ÃƒÂ¨
url         http://www.lascimmiaviaggiatrice.it/isola-di-p...
title       isola di plastica che cos'Ã¨ e perchÃ¨ non puÃ...
position                                                    3
Name: 3, dtype: object

Query                           Quanto psa lisola di plastica
url         http://www.meteoweb.eu/2018/03/inquinamento-oc...
title       Inquinamento: la piÃ¹ grande isola ... - meteo...
position                                                    6
Name: 4, dtype: object

Query                       Dove si trova l isola di plastica
url         http://www.saperescienza.it/rubriche/geologia/...
title       L'isola di plastica - SapereScienza L'isola di..

In [241]:
combined_data = []

for query in click_df_queries:
    search_results = relevance_df.loc[relevance_df["user_query"]==query] # all SE results yielded for a query
    # clicked_results = clicked_df.loc[clicked_df["Query"]==query] # all results clicked for same query
    for _, row in search_results.iterrows():
        session_id = row["session_id"]
        user_query = row["user_query"]
        title = row["title"]
        url = row["url"]
        snippet = row["snippet"]
        textsample = row["TextSample"]
        textsample_tokens = row["TextSample-Tokenized"]
        textsample_pos = row["tagged_TextSample-Tokenized"]
        position = row["position"]
        relevant = row["source"]

        clicked_result_found = clicked_df.loc[(clicked_df["Query"]==query) & (clicked_df["url"]==url) & (clicked_df["title"]==title) & (clicked_df["position"]==position)] # find the clicked result corresponding to (query+url+title+SERP position) combo
        assert len(clicked_result_found) <= 1 
        
        if len(clicked_result_found)==1:
            click = "C"
        elif len(clicked_result_found)==0:
            click="NC"
        
        combined_data.append([session_id, user_query, url, title, snippet, textsample, textsample_tokens, textsample_pos, position, relevant, click])


#     clicked_urls = list(clicked_df.loc[clicked_df["Query"]==query]["url"])
#     resources = relevance_df.loc[relevance_df["user_query"]==query]
#     for _, row in resources.iterrows():
#         session_id = row["session_id"]
#         title = row["title"]
#         snippet = row["snippet"]
#         textsample = row["TextSample"]
#         position = row["position"]
#         relevant = row["source"]
#         url = row["url"]
#         if url in clicked_urls:
#             click = "C"
#         else:
#             click = "NC"

#         combined_data.append([session_id, query,url,title,snippet,textsample, position, relevant, click])
    

combined_data_df = pd.DataFrame(combined_data, columns = ["session_id", "query", "url", "title", "snippet", "text_sample", "text_sample_tokens", "text_sample_pos", "position", "relevant", "click"])
# combined_data_df.drop(["position"], axis=1, inplace=True)
combined_data_df.drop_duplicates(inplace=True, ignore_index=True)
combined_data_df.to_excel("./data/query_log_IT.xlsx", index=False)
print(len(combined_data_df))
combined_data_df.head(2)

670


,session_id,query,url,title,snippet,text_sample,text_sample_tokens,text_sample_pos,position,relevant,click
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",1,R,NC
1,393,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,La deforestazione in Amazzonia - Foresta Amazz...,"['La', 'deforestazione', 'in', 'Amazzonia', '-...","[('La', 'NNP'), ('deforestazione', 'NN'), ('in...",2,NR,NC


In [242]:
len(combined_data_df["query"].unique())

70

In [243]:
combined_data_df["query"].value_counts()

query
PerchÃƒÂ¨ gli orsi polari stanno diminuendo drasticamente di numero    20
Spiagge con bandiera blu                                               18
perchÃƒÂ© gli orsi polari si stanno estinguendo                        18
Anno con temperature piÃƒÂ¹ alte                                       12
Dove si trova Ã„Â¾isola di plastica?                                   11
                                                                       ..
Le regole per tenere le spiagge pulite                                  7
Isola di plastica                                                       6
Conseguenze negative della distruzione della foresta amazzonica         6
perchÃƒÂ© gli orsi polari si stanno estinguendo sito babbanas           6
ocean cleanup                                                           5
Name: count, Length: 70, dtype: int64

In [244]:
combined_data_df.groupby(["click", "relevant"]).size()

click  relevant
C      NR            1
       R            86
NC     NR          576
       R             7
dtype: int64

In [251]:
df = pd.DataFrame(columns=relevance_df.columns)

for _, row in clicked_df.iterrows():
    query = row["Query"]
    title = row["title"]
    url = row["url"]
    position = row["position"]

    result = relevance_df.loc[(relevance_df["user_query"]==query) & (relevance_df["title"]==title) & (relevance_df["url"]==url) & (relevance_df["position"]==position)]
    if len(result) == 0:
        print("No search result corresponding to ", (query, title, url, position), " combination.")
    else:
        df = pd.concat([df, result])


No search result corresponding to  ('BiodiversitÃƒÂ¡', 'Why Is Biodiversity Important? Who Cares? â€” Global Issues', 'http://www.globalissues.org/article/170/why-is-biodiversity-important-who-cares', 9)  combination.
No search result corresponding to  ('Lisola di plastica cosa ÃƒÂ¨', "isola di plastica che cos'Ã¨ e perchÃ¨ non puÃ² essere salvata", 'http://www.lascimmiaviaggiatrice.it/isola-di-plastica-che-cose/', 3)  combination.
No search result corresponding to  ('Quanto psa lisola di plastica', 'Inquinamento: la piÃ¹ grande isola ... - meteoweb.eu', 'http://www.meteoweb.eu/2018/03/inquinamento-oceani-isola-plastica/1070775/', 6)  combination.
No search result corresponding to  ('Dove si trova l isola di plastica', "L'isola di plastica - SapereScienza L'isola di plastica - SapereScienza L'isola di plastica - SapereScienza", 'http://www.saperescienza.it/rubriche/geologia/l-isola-di-plastica-18-07-2014/200-l-isola-di-plastica-18-07-2014', 7)  combination.
No search result correspondi

In [253]:
df["source"].value_counts()

source
R     86
NR     1
Name: count, dtype: int64

## Misc

In [38]:
combined_data_excluding_weird_queries = []

for query in click_df_queries:
    if query in weird_queries: # queries within the same session with multiple snippets corresponding to the same web title
        continue
    search_results = relevance_df.loc[relevance_df["user_query"]==query] # all SE results yielded for a query
    clicked_results = clicked_df.loc[clicked_df["Query"]==query] # all results clicked for a query
    for _, row in search_results.iterrows():
        session_id = row["session_id"]
        user_query = row["user_query"]
        title = row["title"]
        url = row["url"]
        snippet = row["snippet"]
        textsample = row["TextSample"]
        textsample_tokens = row["TextSample-Tokenized"]
        textsample_pos = row["tagged_TextSample-Tokenized"]
        position = row["position"]
        relevant = row["source"]

        clicked_result_found = clicked_results.loc[(clicked_results["url"]==url)&(clicked_results["title"]==title)&(clicked_results["position"]==position)] # find the clicked result corresponding to (query+url+title+SERP position) combo
        assert len(clicked_result_found) <= 1 
        
        if len(clicked_result_found)==1:
            click = "C"
        elif len(clicked_result_found)==0:
            click="NC"
        
        combined_data_excluding_weird_queries.append([session_id, query,url,title,snippet,textsample,textsample_tokens, textsample_pos, position, relevant, click])


#     clicked_urls = list(clicked_df.loc[clicked_df["Query"]==query]["url"])
#     resources = relevance_df.loc[relevance_df["user_query"]==query]
#     for _, row in resources.iterrows():
#         session_id = row["session_id"]
#         title = row["title"]
#         snippet = row["snippet"]
#         textsample = row["TextSample"]
#         position = row["position"]
#         relevant = row["source"]
#         url = row["url"]
#         if url in clicked_urls:
#             click = "C"
#         else:
#             click = "NC"

#         combined_data.append([session_id, query,url,title,snippet,textsample, position, relevant, click])
    

combined_data_excluding_weird_queries_df = pd.DataFrame(combined_data_excluding_weird_queries, columns = ["session_id", "query", "url", "title", "snippet", "text_sample", "text_sample_tokens", "text_sample_pos", "position", "relevant", "click"])
# combined_data_df.drop(["position"], axis=1, inplace=True)
combined_data_excluding_weird_queries_df.drop_duplicates(inplace=True, ignore_index=True)
combined_data_excluding_weird_queries_df.to_excel("./data/query_log_IT_excluding_weird_queries.xlsx", index=False)
print(len(combined_data_excluding_weird_queries_df))
combined_data_excluding_weird_queries_df.head(2)

453


,session_id,query,url,title,snippet,text_sample,text_sample_tokens,text_sample_pos,position,relevant,click
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",1,R,NC
1,393,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,La deforestazione in Amazzonia - Foresta Amazz...,"['La', 'deforestazione', 'in', 'Amazzonia', '-...","[('La', 'NNP'), ('deforestazione', 'NN'), ('in...",2,NR,NC


In [39]:
len(combined_data_excluding_weird_queries_df["query"].unique())

44

In [40]:
combined_data_excluding_weird_queries_df["query"].value_counts()

query
PerchÃƒÂ¨ gli orsi polari stanno diminuendo drasticamente di numero      20
Spiagge con bandiera blu                                                 18
Anno con temperature piÃƒÂ¹ alte                                         12
Dove si trova l isola di plastica                                        12
le conseguenze negative della distruzione della foresta amazzonica       10
spiagge bandiera blu 2019                                                10
CosÃƒÂ© il buco nel azono                                                10
Nomi spiagge bandiera pulizia                                            10
Temperature estive superano quelle misurate nel 2019                     10
Regole per tenere pulite le spiagge                                      10
Le spiagge piÃ…Â« piÃƒÂ¹ pulite                                          10
Peso bottiglia di plastica                                               10
Spiagge con la bandiera della pulizia                                    10
Cose i

In [41]:
combined_data_excluding_weird_queries_df.groupby(["click", "relevant"]).size()

click  relevant
C      R            66
NC     NR          380
       R             7
dtype: int64

In [216]:
count_clicked_urls_missed = 0

for _, row in combined_data_df.iterrows():
    result_url = row["url"]
    click_label = row["click"]
    if click_label == "C":
        if len(clicked_df.loc[clicked_df["url"]==result_url])==0:
            print(row)
    if row["click"]=="NC":
        if len(clicked_df.loc[clicked_df["url"]==result_url])!=0:
            print(result_url, row["session_id"])
            print(list(combined_data_df.loc[combined_data_df["url"]==result_url]["session_id"]))
#             count_clicked_urls_missed+=1

# print(count_clicked_urls_missed)

https://www.ecoage.it/conseguenze-della-deforestazione.htm 393
[393, 417]
https://forestaamazzonica.blogspot.com/2016/08/deforestazione-amazzonia.html 393
[393, 417, 417]
https://it.wikipedia.org/wiki/Pacific_Trash_Vortex 401
[401, 398, 395, 404, 404, 401, 401, 499]
http://www.saperescienza.it/rubriche/geologia/l-isola-di-plastica-18-07-2014/200-l-isola-di-plastica-18-07-2014 401
[401, 395, 395, 404, 401, 401]
http://www.ilbullone.org/lisola-di-plastica/ 401
[401, 395, 499, 499]
https://it.wikipedia.org/wiki/Pacific_Trash_Vortex 398
[401, 398, 395, 404, 404, 401, 401, 499]
http://www.rainews.it/dl/rainews/articoli/russia-orsi-polari-14b21631-5bff-48f1-93c1-749824c1c39c.html 481
[481]
http://www.ilbullone.org/lisola-di-plastica/ 395
[401, 395, 499, 499]
https://gosblogsurf.blogspot.com/2011/01/lisola-di-plastica.html 395
[395, 499]
https://it.answers.yahoo.com/question/index?qid=20090218083835AAVMgRL 411
[411, 435]
https://www.elledecor.com/it/lifestyle/a28066697/incendi-foresta-amazzon

In [60]:
import pandas as pd

df = pd.read_excel("./data/query_log_IT.xlsx")
df.head()

,query,url,title,snippet,text_sample,position,relevant,click
0,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,Conseguenze della deforestazione - ecoage.it C...,1,R,NC
1,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,La deforestazione in Amazzonia - Foresta Amazz...,2,NR,NC
2,le conseguenze negative della distruzione dell...,https://www.elledecor.com/it/lifestyle/a280666...,Le conseguenze degli incendi della foresta ama...,Il Brasile sta vivendo la piÃ¹ grande ondata d...,Le conseguenze degli incendi della foresta ama...,3,R,C
3,le conseguenze negative della distruzione dell...,https://www.informazioneambiente.it/deforestaz...,Cause e conseguenze della deforestazione ambie...,Le conseguenze della deforestazione. Ma a cosa...,Cause e conseguenze della deforestazione ambie...,4,NR,NC
4,le conseguenze negative della distruzione dell...,http://www.eniscuola.net/argomento/foresta-tro...,Conseguenze della deforestazione - eniscuola.net,Conseguenze della deforestazione. Le foreste h...,Conseguenze della deforestazione - eniscuola.n...,5,R,C


In [62]:
len(df["query"].unique()), len(df["text_sample"].unique()), len(df)

(70, 536, 711)

In [ ]:
import pandas as pd



## Final run

In [399]:
import pandas as pd

oct_7_relevance = pd.read_csv("./data_new/QL_IT_oct_7_exp_on_results.csv")
oct_8_relevance = pd.read_csv("./data_new/QL_IT_oct_8_exp_on_results.csv")
oct_10_relevance = pd.read_csv("./data_new/QL_IT_oct_10_exp_on_results.csv")

oct_7_clicks = pd.read_excel("./data_new/QL_Oct 7_clicked.xlsx")
oct_8_clicks = pd.read_excel("./data_new/QL_Oct 8_clicked.xlsx")
oct_10_clicks = pd.read_excel("./data_new/QL_Oct 10_clicked.xlsx")

In [400]:
for session_id in oct_7_relevance["session_id"].unique():
    for query in oct_7_relevance.loc[oct_7_relevance["session_id"]==session_id]["user_query"].unique():
        if len(oct_7_relevance.loc[(oct_7_relevance["user_query"]==query)&(oct_7_relevance["session_id"]==session_id)]) > 10:
            print(session_id, query)

In [401]:
def strip_content(df, clicks=False):
    if clicks == True:
        columns = ["title", "url"]
    else:
        # df = df[["session_id", "user_query", "url", "source", "title", "snippet", "position", "TextSample"]]
        columns = ["url", "TextSample", "title", "snippet"]

    for col in columns:
        df[col] = [val.strip() for val in df[col]]

    return df

In [402]:
oct_7_relevance = strip_content(oct_7_relevance)
oct_8_relevance = strip_content(oct_8_relevance)
oct_10_relevance = strip_content(oct_10_relevance)

oct_7_clicks = strip_content(oct_7_clicks, clicks=True)
oct_8_clicks = strip_content(oct_8_clicks, clicks=True)
oct_10_clicks = strip_content(oct_10_clicks, clicks=True)

In [403]:
for session_id in oct_7_relevance["session_id"].unique():
    for query in oct_7_relevance.loc[oct_7_relevance["session_id"]==session_id]["user_query"].unique():
        if len(oct_7_relevance.loc[(oct_7_relevance["user_query"]==query)&(oct_7_relevance["session_id"]==session_id)]) > 10:
            print(session_id, query)

In [404]:
oct_7_relevance_cleaned = oct_7_relevance.drop_duplicates(ignore_index=True)
oct_7_relevance_cleaned.reset_index(drop=True, inplace=True)

oct_8_relevance_cleaned = oct_8_relevance.drop_duplicates(ignore_index=True)
oct_8_relevance_cleaned.reset_index(drop=True, inplace=True)

oct_10_relevance_cleaned = oct_10_relevance.drop_duplicates(ignore_index=True)
oct_10_relevance_cleaned.reset_index(drop=True, inplace=True)

In [405]:
print(len(oct_7_relevance), len(oct_7_relevance_cleaned))
print(len(oct_8_relevance), len(oct_8_relevance_cleaned))
print(len(oct_10_relevance), len(oct_10_relevance_cleaned))

444 444
218 218
195 195


In [406]:
oct_7_relevance_cleaned.head()

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized,...,E_anticipation,E_disgust,E_fear,E_joy,E_sadness,E_surprise,E_trust,S_pos,S_neg,S_obj
0,393,le conseguenze negative della distruzione del...,https://www.ecoage.it/conseguenze-della-defore...,R,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,1,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",...,0.0,0.20,0.20,0.0,0.20,0.0,0.0,0.000000,0.250000,0.750000
1,393,le conseguenze negative della distruzione del...,https://forestaamazzonica.blogspot.com/2016/08...,NR,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,2,La deforestazione in Amazzonia - Foresta Amazz...,"['La', 'deforestazione', 'in', 'Amazzonia', '-...","[('La', 'NNP'), ('deforestazione', 'NN'), ('in...",...,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.285714,0.714286
2,393,le conseguenze negative della distruzione del...,https://www.elledecor.com/it/lifestyle/a280666...,R,Le conseguenze degli incendi della foresta ama...,Il Brasile sta vivendo la più grande ondata di...,3,Le conseguenze degli incendi della foresta ama...,"['Le', 'conseguenze', 'degli', 'incendi', 'del...","[('Le', 'NNP'), ('conseguenze', 'NN'), ('degli...",...,0.0,0.20,0.20,0.0,0.20,0.2,0.0,0.000000,0.166667,0.833333
3,393,le conseguenze negative della distruzione del...,https://www.informazioneambiente.it/deforestaz...,NR,Cause e conseguenze della deforestazione ambie...,Le conseguenze della deforestazione. Ma a cosa...,4,Cause e conseguenze della deforestazione ambie...,"['Cause', 'e', 'conseguenze', 'della', 'defore...","[('Cause', 'NNP'), ('e', 'NN'), ('conseguenze'...",...,0.0,0.25,0.25,0.0,0.25,0.0,0.0,0.166667,0.250000,0.583333
4,393,le conseguenze negative della distruzione del...,http://www.eniscuola.net/argomento/foresta-tro...,R,Conseguenze della deforestazione - eniscuola.net,Conseguenze della deforestazione. Le foreste h...,5,Conseguenze della deforestazione - eniscuola.n...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",...,0.0,0.00,0.00,0.0,0.00,0.0,1.0,0.333333,0.000000,0.666667


In [408]:
oct_7_relevance_cleaned.to_csv("./data_new/oct_7_relevance_cleaned.csv", index=False)
oct_8_relevance_cleaned.to_csv("./data_new/oct_8_relevance_cleaned.csv", index=False)
oct_10_relevance_cleaned.to_csv("./data_new/oct_10_relevance_cleaned.csv", index=False)

In [409]:
def check_for_duplicates(df):
    for session_id in df["session_id"].unique():
        for query in df.loc[df["session_id"]==session_id]["user_query"].unique():
            if len(df.loc[(df["session_id"]==session_id) & (df["user_query"]==query)]) != len(df.loc[(df["session_id"]==session_id) & (df["user_query"]==query)]["TextSample"].unique()):
                for textsample in df.loc[(df["session_id"]==session_id) & (df["user_query"]==query)]["TextSample"].unique():
                    if len(df.loc[(df["session_id"]==session_id) & (df["user_query"]==query) & (df["TextSample"]==textsample)]) > 1:
                        print(session_id)
                        print("\t", query)
                        print("\t\t", textsample, df.loc[(df["session_id"]==session_id) & (df["user_query"]==query) & (df["TextSample"]==textsample)].index)

In [410]:
len(oct_7_relevance_cleaned["session_id"].unique())

11

In [411]:
len(oct_7_relevance_cleaned)

444

In [412]:
check_for_duplicates(df=oct_7_relevance_cleaned)

395
	 Le regole per tenere le spiagge pulite
		 Spiagge italiane: ecco le regole per trattarle bene | Ecoo. Spiagge italiane: ecco le regole per trattarle bene. Diverse associazioni ambientaliste si sono impegnate nel mettere a punto uno specifico decalogo per le spiagge pulite, in vista di un impatto ambientale minore sull’ecosistema marino Index([52, 53], dtype='int64')
398
	 BiodiversitÃ¡
		 Why Is Biodiversity Important? Who Cares? — Global Issues. Biodiversity is important, more than just the 'I want my children to enjoy it' reason. For example, the richness of diversity allows medicines and foods to be naturally available. The natural disaster prevention mechanisms in most ecosystems and other free services we all get from the surrounding environment are not easily replaceable or replicable, so maintaining biodiversity is important. Index([124, 125], dtype='int64')
404
	 Isola di plastica
		 Isola di plastica: il simbolo di un Pianeta che sta morendo. Ultimamente si è parlato spe

In [413]:
check_for_duplicates(df=oct_8_relevance_cleaned)

433
	 netflix
		 Netflix. Watch TV shows and movies anytime, anywhere. Only $7.99 a month. Start your free month. Index([30, 31], dtype='int64')
438
	 Cosa Ã¨ ocean cleanup? Ita
		 Ocean Cleanup, parte progetto per pulire l’oceano dalla .... Il dispositivo galleggiante realizzato per raccogliere i rifiuti dall’Oceano finalmente funziona. Ad annunciarlo, attraverso un tweet, è stato lo stesso Boyan Slat, ovvero colui che l’ha ideato quando aveva solo 18 anni, e che ha fondato il progetto Ocean Cleanup, l’organizzazione senza scopro di lucro il cui obiettivo è appunto la pulizia degli oceani. Index([119, 122], dtype='int64')


In [414]:
check_for_duplicates(df=oct_10_relevance_cleaned)

466
	 A cosa serve frigorifero solidale
		 Il frigorifero solidale di Dounia che offre cibo a chi ne .... Il frigorifero solidale di Dounia che offre cibo a chi ne ha bisogno A metà fra Montmartre e Barbès, una ragazza apre il suo frigo a chi altrimenti rovisterebbe nella spazzatura. Index([25, 27], dtype='int64')


In [416]:
len(oct_7_relevance_cleaned)+len(oct_8_relevance_cleaned)+len(oct_10_relevance_cleaned)

857

In [293]:
import pandas as pd

sample_relevance_csv = pd.read_csv("./data_new/sample_relevance.csv")
sample_relevance_excel = pd.read_excel("./data_new/sample_relevance.xlsx")
sample_clicks = pd.read_excel("./data_new/sample_clicks.xlsx")

In [294]:
sample_relevance_csv["url"] = [val.strip() for val in sample_relevance_csv["url"]]

In [295]:
sample_relevance_excel["url"] = [val.strip() for val in sample_relevance_excel["url"]]

In [296]:
sample_clicks["url"] = [val.strip() for val in sample_clicks["url"]]

In [297]:
sample_relevance_csv.loc[sample_relevance_csv["url"].isin(sample_clicks["url"])]

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized,...,E_anticipation,E_disgust,E_fear,E_joy,E_sadness,E_surprise,E_trust,S_pos,S_neg,S_obj
20,395,I ghiacciai in via di estinzione,https://trekking.it/news/ghiacciai-specie-in-e...,R,"Ghiacciai, specie in estinzione | Trekking.it",Un esempio per tutti: la Groenlandia rappresen...,1,"Ghiacciai, specie in estinzione | Trekking.it....","['Ghiacciai', ',', 'specie', 'in', 'estinzione...","[('Ghiacciai', 'NNP'), (',', ','), ('specie', ...",...,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.117647,0.058824,0.823529
41,395,Le regole per tenere le spiagge pulite,https://www.105.net/gallery/tutto-news/249890/...,NR,10 consigli per mantenere le spiagge pulite - ...,10 consigli per mantenere le spiagge pulite Pe...,2,10 consigli per mantenere le spiagge pulite - ...,"['10', 'consigli', 'per', 'mantenere', 'le', '...","[('10', 'CD'), ('consigli', 'NN'), ('per', 'IN...",...,0.2,0.0,0.0,0.2,0.0,0.2,0.4,0.125000,0.000000,0.875000
51,395,Le regole per tenere le spiagge pulite,https://www.105.net/gallery/tutto-news/249890/...,R,10 consigli per mantenere le spiagge pulite - ...,Canale 105 > News > Tutto News; tutto news. 03...,2,10 consigli per mantenere le spiagge pulite - ...,"['10', 'consigli', 'per', 'mantenere', 'le', '...","[('10', 'CD'), ('consigli', 'NN'), ('per', 'IN...",...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,1.000000


In [299]:
sample_relevance_excel.loc[sample_relevance_excel["url"].isin(list(sample_clicks["url"]))]

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized,...,E_anticipation,E_disgust,E_fear,E_joy,E_sadness,E_surprise,E_trust,S_pos,S_neg,S_obj
20,395,I ghiacciai in via di estinzione,https://trekking.it/news/ghiacciai-specie-in-e...,R,"Ghiacciai, specie in estinzione | Trekking.it",Un esempio per tutti: la Groenlandia rappresen...,1,"Ghiacciai, specie in estinzione | Trekking.it....","['Ghiacciai', ',', 'specie', 'in', 'estinzione...","[('Ghiacciai', 'NNP'), (',', ','), ('specie', ...",...,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.117647,0.058824,0.823529
41,395,Le regole per tenere le spiagge pulite,https://www.105.net/gallery/tutto-news/249890/...,NR,10 consigli per mantenere le spiagge pulite - ...,10 consigli per mantenere le spiagge pulite Pe...,2,10 consigli per mantenere le spiagge pulite - ...,"['10', 'consigli', 'per', 'mantenere', 'le', '...","[('10', 'CD'), ('consigli', 'NN'), ('per', 'IN...",...,0.2,0.0,0.0,0.2,0.0,0.2,0.4,0.125000,0.000000,0.875000
51,395,Le regole per tenere le spiagge pulite,https://www.105.net/gallery/tutto-news/249890/...,R,10 consigli per mantenere le spiagge pulite - ...,Canale 105 > News > Tutto News; tutto news. 03...,2,10 consigli per mantenere le spiagge pulite - ...,"['10', 'consigli', 'per', 'mantenere', 'le', '...","[('10', 'CD'), ('consigli', 'NN'), ('per', 'IN...",...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,1.000000


In [300]:
for url in sample_clicks["url"]:
    if url in list(sample_relevance_csv["url"]):
        print(url)

https://trekking.it/news/ghiacciai-specie-in-estinzione.html
https://www.105.net/gallery/tutto-news/249890/10-consigli-per-mantenere-le-spiagge-pulite.html


In [301]:
for url in sample_clicks["url"]:
    if url in list(sample_relevance_excel["url"]):
        print(url)

https://trekking.it/news/ghiacciai-specie-in-estinzione.html
https://www.105.net/gallery/tutto-news/249890/10-consigli-per-mantenere-le-spiagge-pulite.html
